In [1]:
import numpy as np


In [2]:
def get_kernel_size(factor):
    #
    return 2*factor-factor%2

In [3]:
def upsample_filt(size):
    #
    factor=(size+1)//2
    if size %2==1:
        center=factor-1
    else:
        center=factor-0.5
    og=np.ogrid[:size,:size]
    return(1-abs(og[0]-center)/factor)*\
          (1-abs(og[1]-center)/factor)

In [4]:
def bilinear_upsample_weights(factor,number_of_classes):
    filter_size=get_kernel_size(factor)
    weights=np.zeros((filter_size,
                    filter_size,
                    number_of_classes,
                    number_of_classes),dtype=np.float32)
    upsample_kernel=upsample_filt(filter_size)
    for i in range(number_of_classes):
        #!!!!!
        weights[:,:,i,i]=upsample_kernel
    return weights

In [5]:
%matplotlib inline

from __future__ import division
import os
import sys
import tensorflow as tf
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

slim=tf.contrib.slim
sys.path.append(os.path.expanduser("../../week8/models-r1.5/research/slim/"))
from nets import vgg
from preprocessing import vgg_preprocessing

c:\users\shenchao\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "c:\users\shenchao\appdata\local\programs\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\shenchao\appdata\local\programs\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\shenchao\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\shenchao\appdata\local\programs\python\python36\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance


In [6]:
os.environ["CUDA_DEVICE_ORDER"]="00000000:01:00.0"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#%env CUDA_DEVICE_ORDER=00000000:01:00.0
#%env CUDA_VISIBLE_DEVICES=0
checkpoints_dir=os.path.expanduser('data/pre_trained/')
image_filename='object_2.jpg'
annotation_filename='segment_2.png'
fig_size=[15,4]
plt.rcParams["figure.figsize"]=fig_size

In [7]:
tf.reset_default_graph()
image_filename_placeholder=tf.placeholder(tf.string)
annotation_filename_placeholder=tf.placeholder(tf.string)
is_training_placeholder=tf.placeholder(tf.bool)
feed_dict_to_use={image_filename_placeholder:image_filename,
                 annotation_filename_placeholder:annotation_filename,
                 is_training_placeholder:True}
image_tensor=tf.read_file(image_filename_placeholder)
annotation_tensor=tf.read_file(annotation_filename_placeholder)

image_tensor=tf.image.decode_jpeg(image_tensor,channels=3)
annotation_tensor=tf.image.decode_png(annotation_tensor,channels=1)

class_labels_tensor=tf.greater_equal(annotation_tensor,1)
background_labels_tensor=tf.less(annotation_tensor,1)

bit_mask_class=tf.to_float(class_labels_tensor)
bit_mask_background=tf.to_float(background_labels_tensor)

combined_mask=tf.concat(axis=2,values=[bit_mask_background,
                                      bit_mask_class,])
tensor=combined_mask
flat_labels=tf.reshape(tensor=combined_mask,shape=[-1,2])
#!!!!!!!!!!!!!!!!!

In [8]:
from preprocessing.vgg_preprocessing import (_mean_image_subtraction,
                                            _R_MEAN,_G_MEAN,_B_MEAN)
upsample_factor=32
number_of_classes=2
log_folder=os.path.expanduser('segment_log_folder')
vgg_checkpoint_path=os.path.join(checkpoints_dir,'vgg_16.ckpt')

image_float=tf.to_float(image_tensor,name='ToFloat')
original_shape=tf.shape(image_float)[0:2]

mean_centered_image=_mean_image_subtraction(image_float,
                                           [_R_MEAN,_G_MEAN,_B_MEAN])
target_input_size_factor=tf.ceil(tf.div(tf.to_float(original_shape),
                                       tf.to_float(upsample_factor)))
target_input_size=tf.to_int32(tf.multiply(target_input_size_factor,upsample_factor))
padding_size=(target_input_size-original_shape)//2

mean_centered_image=tf.image.pad_to_bounding_box(mean_centered_image,
                                                padding_size[0],
                                                padding_size[1],
                                                target_input_size[0],
                                                target_input_size[1])

processed_images=tf.expand_dims(mean_centered_image,0)

upsample_filter_np=bilinear_upsample_weights(upsample_factor,
                                             number_of_classes)

upsample_filter_tensor=tf.Variable(upsample_filter_np,name='vgg_16/fc8/t_conv')

with slim.arg_scope(vgg.vgg_arg_scope()):
    logits,end_points=vgg.vgg_16(processed_images,
                                 num_classes=2,
                              
                                 is_training=is_training_placeholder,
                              
                                 spatial_squeeze=False,
                              
                                 fc_conv_padding='SAME')

downsampled_logits_shape=tf.shape(logits)

upsampled_logits_shape=tf.stack([downsampled_logits_shape[0],
                                 original_shape[0],
                                 original_shape[1],
                                 downsampled_logits_shape[3]
                                 ])
upsampled_logits=tf.nn.conv2d_transpose(logits,upsample_filter_tensor,
                                        output_shape=upsampled_logits_shape,
                                        strides=[1,upsample_factor,upsample_factor,1],
                                        padding='SAME'
                                       )


flat_logits=tf.reshape(tensor=upsampled_logits,shape=(-1,number_of_classes))

cross_entropies=tf.nn.softmax_cross_entropy_with_logits(logits=flat_logits,
                                                       labels=flat_labels)

cross_entropy_sum=tf.reduce_sum(cross_entropies)

pred=tf.argmax(upsampled_logits,axis=3)

probabilities=tf.nn.softmax(upsampled_logits)

with tf.variable_scope("adam_vars"):
    optimizer=tf.train.AdamOptimizer(learning_rate=1e-4)
    gradients=optimizer.compute_gradients(loss=cross_entropy_sum)
    for grad_var_pair in gradients:
        current_variable=grad_var_pair[1]
        current_gradient=grad_var_pair[0]
        gradient_name_to_save=current_variable.name.replace(":","_")
        tf.summary.histogram(gradient_name_to_save,current_gradient)
    train_step=optimizer.apply_gradients(grads_and_vars=gradients)
vgg_except_fc8_weights=slim.get_variables_to_restore(exclude=['vgg_16/fc8','adam_vars'])
vgg_fc8_weights=slim.get_variables_to_restore(include=['vgg_16/fc8'])
adam_optimizer_variables=slim.get_variables_to_restore(include=['adam_vars'])
tf.summary.scalar('cross_entropy_loss',cross_entropy_sum)
merged_summary_op=tf.summary.merge_all()
summary_string_writer=tf.summary.FileWriter(log_folder)
if not os.path.exists(log_folder):
    os.makedirs(log_folder)
read_vgg_weights_except_fc8_func=slim.assign_from_checkpoint_fn(vgg_checkpoint_path,
                                                                vgg_except_fc8_weights)

vgg_fc8_weights_initializer=tf.variables_initializer(adam_optimizer_variables)

optimization_variables_initializer=tf.variables_initializer(adam_optimizer_variables)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [ ]:
init_op=tf.global_variables_initializer()

sess_config=tf.ConfigProto()
sess_config.gpu_options.allow_growth=True
sess= tf.Session(config=sess_config)
with sess:
    sess.run(tf.global_variables_initializer())
    read_vgg_weights_except_fc8_func(sess)    
    sess.run(vgg_fc8_weights_initializer)
    sess.run(optimization_variables_initializer)

    train_image,train_annotation=sess.run([image_tensor,annotation_tensor],
                                         feed_dict=feed_dict_to_use)
    f,(ax1,ax2)=plt.subplots(1,2,sharey=True)
    ax1.imshow(train_image)
    ax1.set_title("input image")
    probability_graph=ax2.imshow(np.dstack((train_annotation,)*3)*100)
    ax2.set_title("input ground truth annotation")
    plt.show()
    downsample_logits_value,train_annotation=sess.run([downsampled_logits_shape,annotation_tensor],feed_dict=feed_dict_to_use)
    print(downsampled_logits_shape.shape)
    
    for i in range(30):
        print(i)
        loss,summary_string=sess.run([cross_entropy_sum,merged_summary_op],
                                    feed_dict=feed_dict_to_use)
        
        sess.run(train_step,feed_dict=feed_dict_to_use)
        
        pred_np,probabilities_np=sess.run([pred,probabilities],
                                         feed_dict=feed_dict_to_use)
        
        summary_string_writer.add_summary(summary_string,i)
        
        #camp='seismic'                 
        f,(ax1,ax2,ax3)=plt.subplots(1,3,sharey=True)
        ax1.imshow(np.uint8(pred_np.squeeze()!=1),vmax=1.5,vmin=-0.4)
        ax1.set_title('Arg max.interation #'+str(i))
        probability_graph=ax2.imshow(probabilities_np.squeeze()[:,:,0])
        ax2.set_title('probability of the class.Interation # '+str(i))
        mask=np.multiply(np.uint32(pred_np.squeeze()),128)
        mask=np.stack([mask,]*3,axis=-1)
        masked_image=np.uint8(np.clip(train_image+mask,0,255))
        probability_graph=ax3.imshow(masked_image)
        plt.colorbar(probability_graph)
        plt.show()
        print("current loss:"+str(loss))
    feed_dict_to_use[is_training_placeholder]=False
    final_predictions,final_probabilities,final_loss=sess.run([pred,
                                                              probabilities,
                                                              cross_entropy_sum],
                                                             feed_dict=feed_dict_to_use)
    f,(ax1,ax2,ax3)=plt.subplots(1,3,sharey=True)
    ax1.imshow(np.uint8(final_predictions.squeeze()!=1),
               vmax=1.5,
               vmin=-0.4
               )
    ax1.set_title('final argmax')
    probability_graph=ax2.imshow(final_probabilities.squeeze()[:,:,0])
    plt.colorbar(probability_graph)
    mask=np.multiply(np.uint32(final_predictions.squeeze()),128)
    mask=np.stack([np.zeros(mask.shape),
               mask,
               np.zeros(mask.shape)],axis=-1)
    masked_image=np.uint(np.clip(train_image+mask,0,255))
    probability_graph=ax3.imshow(masked_image)
    plt.show()
    print("fina loss:",str(final_loss))
summary_string_writer.close()

In [9]:
import pydensecrf.densecrf as dcrf 

In [10]:
import sys

#path = "/home/dpakhom1/dense_crf_python/"
#sys.path.append(path)

import pydensecrf.densecrf as dcrf

from pydensecrf.utils import compute_unary, create_pairwise_bilateral, \
    create_pairwise_gaussian, softmax_to_unary

import skimage.io as io

image = train_image
processed_probabilities=final_probabilities.squeeze()
softmax = processed_probabilities.transpose((2, 0, 1))

# 输入数据应为概率值的负对数
# 你可以在softmax_to_unary函数的定义中找到更多信息
unary = softmax_to_unary(softmax)
print(unary.shape)

# 输入数据应为C-连续的——我们使用了Cython封装器
unary = np.ascontiguousarray(unary)

d = dcrf.DenseCRF(image.shape[0] * image.shape[1], 2)

d.setUnaryEnergy(unary)

# 潜在地对空间上相邻的小块分割区域进行惩罚——促使产生更多空间连续的分割区域
feats = create_pairwise_gaussian(sdims=(10, 10), shape=image.shape[:2])

d.addPairwiseEnergy(feats, compat=3,
                    kernel=dcrf.DIAG_KERNEL,
                    normalization=dcrf.NORMALIZE_SYMMETRIC)

# 这将创建与颜色相关的图像特征——因为我们从卷积神经网络中得到的分割结果非常粗糙，
# 我们可以使用局部的颜色特征来改善分割结果
feats = create_pairwise_bilateral(sdims=(50, 50), schan=(20, 20, 20),
                                   img=image, chdim=2)

d.addPairwiseEnergy(feats, compat=10,
                     kernel=dcrf.DIAG_KERNEL,
                     normalization=dcrf.NORMALIZE_SYMMETRIC)
Q = d.inference(5)

res = np.argmax(Q, axis=0).reshape((image.shape[0], image.shape[1]))

#cmap = plt.get_cmap('bwr')

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
ax1.imshow(res, vmax=1.5, vmin=-0.4)
ax1.set_title('Segmentation with CRF post-processing')
probability_graph = ax2.imshow(np.dstack((train_annotation,)*3)*100)
ax2.set_title('Ground-Truth Annotation')
mask=np.multiply(np.uint32(res.squeeze()),128)
mask=np.stack([np.zeros(mask.shape),
              mask,
              np.zeros(mask.shape)],axis=-1)
masked_image=np.uint8(np.clip(np.uint32(train_image)+mask,0,255))
probability_graph=ax3.imshow(masked_image)

plt.show()
intersection=np.logical_and(res,train_annotation.squeeze())
union=np.logical_or(res,train_annotation.squeeze())
sum_intersection=np.sum(intersection)
sum_union=np.sum(union)
print('IoU:%.2f%%'%((sum_intersection/sum_union)*100))


ERROR! Session/line number was not unique in database. History logging moved to new session 762


NameError: name 'train_image' is not defined

In [3]:
import gdal
import numpy as np
image_sets = ['test.tif']
data_tif=gdal.Open('./data_MB/' + image_sets[0])
x_width  = data_tif.RasterXSize    #栅格矩阵的列数
x_height = data_tif.RasterYSize
data=data_tif.ReadAsArray(0,0,x_width,x_height)
im_data=np.append(np.append(np.append(data[3,:,:][:,:,np.newaxis],data[2,:,:][:,:,np.newaxis],axis=2),data[1,:,:][:,:,np.newaxis],axis=2),data[0,:,:][:,:,np.newaxis],axis=2)
     

In [5]:
data.shape

(4, 1960, 2505)

In [7]:
im_data.shape

(1960, 2505, 4)

In [10]:
import cv2
label_img_gray = cv2.imread('./data_MB/test_label.png' ,cv2.IMREAD_GRAYSCALE)  
label= cv2.imread('./data_MB/test_label.png' )  

In [9]:
label_img_gray.shape

(1960, 2505)

In [11]:
label.shape

(1960, 2505, 3)